# Containerizing an app

#### Docker is all about making it easy to take application source code and get it running in a container. This process is called containerization.

#### Containers are all about making apps simple to build, ship, and run. The end-to-end process looks like this:

- Start with your application code and dependencies
- Create a Dockerfile that describes your app, dependencies, and how to run it
- Build it into an image by passing the Dockerfile to the docker build command
- Push the new image to a registry (optional)
- Run a container from the image

![Sample Image](/Users/maukanmir/Documents/Machine-Learning/AI-ML-Textbooks/AI-ML-Learning/images/figure8-1.png)

- cat Dockerfile

- FROM alpine
- LABEL maintainer="nigelpoulton@hotmail.com"
- RUN apk add --update nodejs npm
- COPY . /src
- WORKDIR /src
- RUN npm install
- EXPOSE 8080
- ENTRYPOINT ["node", "./app.js"]

#### At a high-level, the example Dockerfile says: Start with the alpine image, make a note that “nigelpoulton@hotmail.com” is the maintainer, install Node.js and NPM, copy everything in the build context to the /src directory in the image, set the working directory as /src, install dependencies, document the app’s network port, and set app.js as the default application to run.

#### Dockerfiles normally start with the FROM instruction. This pulls an image that will be used as the base layer for the image the Dockerfile will build – everything else will be added as new layers above this base layer. The app being defined in this Dockerfile is a Linux app, so it’s important that the FROM instruction refers to a Linux-based image. If you’re containerizing a Windows application, you’ll need to specify an appropriate Windows base image.

#### Next, the Dockerfile creates a LABEL that specifies “nigelpoulton@hotmail.com” as the maintainer of the image. Labels are optional key-value pairs and are a good way of adding custom metadata. It’s considered a best practice to list a maintainer so that other users have a point of contact to report problems etc.

#### The RUN apk add --update nodejs nodejs-npm instruction uses the apk package manager to install nodejs and nodejs-npm into the image. It does this by adding a new layer and installing the packages to this layer.

#### The COPY . /src instruction creates another new layer and copies in the application and dependency files from the build context.

#### Next, the Dockerfile uses the WORKDIR instruction to set the working directory for the rest of the instructions. This creates metadata and does not create a new image layer.

#### The RUN npm install instruction runs within the context of the WORKDIR set in the previous instruction, and installs the dependencies listed in package.json into another new layer. At this point in the Dockerfile the image has four layers as shown in Figure 8.5.

![Sample Image](/Users/maukanmir/Documents/Machine-Learning/AI-ML-Textbooks/AI-ML-Learning/images/figure8-5.png)

#### The application exposes a web service on port 8080, so the Dockerfile documents this with the EXPOSE 8080 instruction. Finally, the ENTRYPOINT instruction sets the application to run when started as a container. Both of these are added as metadata and do not create new layers.

#### The following command will build a new image called ddd-book:ch8.1. The period (.) at the end of the command tells Docker to use the working directory as the build context. Remember, the build context is where the app and all dependencies are stored.
#### Be sure to include the trailing period (.) and be sure to run the command from the web-app directory.

- docker build -t ddd-book:ch8.1 .

## Pushing images
#### Once you’ve created an image, it’s a good idea to store it in a registry to keep it safe and make it available to others. Docker Hub is the most common public image registry and it’s the default push location for docker push commands.
#### You’ll need a Docker ID and if you want to push the image to Docker Hub. You’ll also need to tag the image appropriately.

#### Images need to be appropriately tagged before you can push them. This is because the tag includes the following important registry-related information:

- Registry DNS name
- Repository name
- Tag

- docker tag <current-tag> <new-tag>.

#### Now that the image is pushed to a registry, you can access it from anywhere with an internet connection. You can also grant other people access to pull it and push changes.

#### The following command will start a new container called c1 based on the ddd-book:ch8.1 image you just created. It maps port 80 on the Docker host, to port 8080 inside the container. This means you’ll be able to point a web browser at the DNS name or IP address of the Docker host running the container and access the app.

Note: If your host is already running a service on port 80 you’ll get a port is already allocated error. If this happens, specify a different port such as 5000 or 5001. For example, to map the app to port 5000 on the Docker host, use the -p 5000:8080 flag.

- docker run -d --name c1 -p 80:8080 ddd-book:ch8.1
- The -d flag runs the container in the background, and the -p 80:8080 flag maps port 80 on the host to port 8080 inside the running container.

#### Examples of instructions that create new layers are FROM, RUN, and COPY. Examples that create metadata include EXPOSE, WORKDIR, ENV, and ENTRYPOINT. The basic premise is this — if an instruction adds content such as files and programs, it will create a new layer. If it is adding instructions on how to build the image and run the container, it will create metadata.

#### It’s considered a good practice to use images from official repositories as the base layer for new images. This is because their content is vetted and they are quick to release new versions when vulnerabilities are fixed.

## When it comes to Docker images, big is bad! For example:

- Big means slow
- Big means more potential vulnerabilities
- Big means a bigger attack surface

#### In the past, keeping images small was hard work. However, multi-stage builds make it easy. Here’s the high-level…

#### Multi-stage builds have multiple FROM instructions in a single Dockerfile, and each FROM instruction is a new build stage. You can do the heavy-lifting work of building your app in a large image with all the compilers and other build tools required. You can then copy the final production app into a tiny image used for production. You can also perform build steps in parallel for faster builds.

- FROM golang:1.20-alpine AS base
- WORKDIR /src
- COPY go.mod go.sum .
- RUN go mod download
- COPY . .

- FROM base AS build-client
- RUN go build -o /bin/client ./cmd/client

- FROM base AS build-server
- RUN go build -o /bin/server ./cmd/server

- FROM scratch AS prod
- COPY --from=build-client /bin/client /bin/
- COPY --from=build-server /bin/server /bin/
- ENTRYPOINT [ "/bin/server" ]

#### The first thing to note is that the Dockerfile has four FROM instructions. Each of these is a distinct build stage and Docker numbers them starting from 0. However, each stage has been given a friendly name.
- Stage 0 is called base
- Stage 1 is called build-client
- Stage 2 is called build-server
- Stage 3 is called prod

#### The goal of the base stage is to create a reusable build image with all the tools needed to build the application in the later stages. The image created by this stage will only be used to build the app and not used for production.

#### It pulls the golang:1.20-alpine image which is over 250MB when uncompressed on the host. It sets the working directory to /src and copies in the go.mod and go.sum files. These list the application dependencies and hashes. Next, it installs the dependencies and copies the app code into the image. This stage will add three new layers containing a lot of build stuff but not very much app code. When this build stage is complete it will output a large image that can be used by later stages.

#### The build-client stage doesn’t pull a new image. Instead, it uses the FROM base AS build-client instruction to use the intermediate image created by the base stage. It then uses a RUN instruction to build the client app into a binary executable. The goal of this stage is to create an image with the compiled client binary that can be referenced by later build stages.

#### The build-server stage does the same for the server component of the app and outputs an image that can be referenced by later stages as build-server

#### The build-client and build-server stages can run in parallel, speeding up your build process.

#### The prod stage pulls the minimal scratch image. It then uses the COPY --from instruction to copy the compiled client app from the build-client stage and the compiled server app from the build-server stage. It outputs the final image which is just the client and server apps in a tiny scratch image.

#### The build-server stage does the same for the server component of the app and outputs an image that can be referenced by later stages as build-server

#### The prod stage pulls the minimal scratch image. It then uses the COPY --from instruction to copy the compiled client app from the build-client stage and the compiled server app from the build-server stage. It outputs the final image which is just the client and server apps in a tiny scratch image.

#### The following docker history command shows the final production image with just two layers – one copying in the client binary and the other copying in the server binary. None of the previous build stages are included in this final production image.